In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/lib/python3.6/site-packages/matplotlib/font_manager.py:232: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


### 1. 数据导入

In [2]:
city_road = pd.read_csv('/home/kesci/input/Country_Road5135/city_roads.csv')
taiwan_road = pd.read_csv('/home/kesci/input/Country_Road5135/taiwan_roads.csv')
city_data = pd.read_csv('/home/kesci/input/Country_Road5135/city_data.csv')

In [3]:
city_road.sample(5)

,city,road
39153,广州,YM01-新合路 ...
48584,杭州,荏山路 ...
72967,茂名,广海公路 ...
131063,无锡,沅泾路 ...
19167,常州,潘丰路 ...


In [4]:
city_road['road'] = city_road['road'].str.strip()

In [5]:
city_road.sample(5)

,city,road
34276,佛山,义学街
54185,海口,金濂路
13144,成都,茂达街
39520,广州,Y903-利丰大道
17532,长春,绿园街


In [6]:
taiwan_road.sample(5)

,city,road
6217,云林县,四维街
12664,新竹县,工业四路
15081,彰化县,三成村小西路
27275,云林县,北糖四街
2112,高雄市,溪寮路２段


### 2. 数据清洗
1. 清洗出包含“街”、“道”、“巷”、“路”的道路
2. 增加字段“province”表示城市所在的省份
3. 把台湾的数据合并进来
4. 去重
5. 清除杂乱的字符

#### 2.1 清洗出包含“街”、“道”、“巷”、“路”的道路

In [7]:
    city_road = city_road[(city_road['road'].str.contains(
        '胡同|巷|街|路|道')) | (city_road['road'].str.endswith('里'))]


#### 2.2 增加字段“province_short”表示城市所在的省份

In [8]:
city_data.sample(5)

,city,province
129,和龙,吉林省
326,蓬莱,山东省
232,铜陵,安徽省
281,瑞昌,江西省
531,彭州,四川省


In [9]:
city_data['province'].unique()

array(['河北省', '山西省', '内蒙古自治区', '辽宁省', '吉林省', '黑龙江省', '江苏省', '浙江省', '安徽省',
       '福建省', '江西省', '山东省', '河南省', '湖北省', '湖南省', '广东省', '广西壮族自治区', '海南省',
       '四川省', '贵州省', '云南省', '陕西省', '甘肃省', '青海省', '西藏自治区', '宁夏回族自治区', '台湾',
       '北京市', '天津市', '上海市', '重庆市', '香港', '澳门', '新疆维吾尔自治区'], dtype=object)

In [10]:
city_data['province_short'] = city_data['province'].str.replace(
    "省|市|维吾尔自治区|壮族自治区|回族自治区|自治区", "")

In [11]:
city_road = city_road.merge(city_data[['city', 'province_short']], how='left', on='city')

In [12]:
city_road.sample(5)

,city,road,province_short
70137,南平,杭头街,福建
95101,宿迁,井马线路,江苏
4300,北京,前章村西路,北京
25102,福州,红湖中路,福建
96439,绍兴,雍平支路,浙江


In [13]:
taiwan_road['province_short'] = '台湾'

In [14]:
taiwan_road['city'] = taiwan_road['city'].str.replace('县|市', '')

#### 2.3 把台湾的数据合并进来

In [15]:
data = pd.concat([city_road, taiwan_road])

In [16]:
data.sample(5)

,city,road,province_short
110169,无锡,长安东街,江苏
51468,金华,解放中街,浙江
15671,新北,中山路２段,台湾
43,安阳,阜民街,河南
135340,重庆,平安三路,重庆


#### 2.4 去除重复数据

In [17]:
data.drop_duplicates(inplace=True)

In [18]:
data[data['city']=='上海'].sample(5)

,city,road,province_short
83131,上海,沁春路,上海
78823,上海,奥伟路,上海
84039,上海,天钥桥南路,上海
78820,上海,安东路,上海
82847,上海,南宅路,上海


#### 2.5 清除杂乱的字符

In [19]:
# 去除高速公路、隧道
data = data[-data['road'].str.contains('G|隧道|高速')]

In [20]:
data.sample(5)

,city,road,province_short
53161,江门,德汉街,广东
107784,铜仁,川侗南路,贵州
5267,高雄,镇川五巷,台湾
135291,重庆,南天湖西路,重庆
131978,宜宾,铁肩街,四川


In [21]:
# 去除X***字符
data.loc[data['road'].str.contains('-'), 'road'] = data.loc[data['road'].str.contains('-'), 'road'].str.split('-', expand=True)[1]

In [22]:
data.sample(5)

,city,road,province_short
15119,彰化,斗苑路顶后段,台湾
16321,台东,龙三路,台湾
17078,屏东,龙湾巷,台湾
87032,上海,恒南路,上海
117407,渭南,岳庙东街,陕西


In [23]:
data.shape

(151174, 3)

### 3. 问题
1. 哪些省份/城市在道路名中出镜率最高？
2. 哪些城市热衷于使用其它城市名作为道路名？
3. 街、道、路、巷、胡同、里有何不同？
4. 各城市都有“建设路”吗？还有哪些具有时代特色的路名？
5. 体现人们美好愿望的路名？
6. 体现区域地理的路名？
7. 体现儒释道特色的路名？
8. 体现周围景观的路名？

#### 3.1 哪些省份/城市在道路名中出镜率最高？

In [24]:
city_data.head()

,city,province,province_short
0,石家庄,河北省,河北
1,邯郸,河北省,河北
2,唐山,河北省,河北
3,保定,河北省,河北
4,秦皇岛,河北省,河北


##### 3.1.1 出境最多的省份

In [25]:
province = list(city_data['province_short'].unique())
province_join = '|'.join(province)

In [26]:
data['road'].str.extract(r"({})".format(province_join))[0].value_counts()

山东     319
山西     281
河南     230
河北     218
江西     156
湖南     130
湖北     126
北京      62
海南      52
上海      50
云南      45
重庆      42
天津      39
吉林      33
陕西      30
青海      27
广东      27
宁夏      23
四川      23
香港      20
福建      19
贵州      18
辽宁      16
黑龙江     15
浙江      14
广西      14
江苏      14
新疆      10
澳门      10
甘肃       8
西藏       8
台湾       6
安徽       5
内蒙古      1
Name: 0, dtype: int64

In [27]:
# 探索了一圈，发现很多是以“X山东X”这种格式的，并不是以山东省名称命名，只是恰好路名里带“山东”两个字
# 其它包括山西、河南、河北、江西、湖南、湖北、海南等都是如此，所以还得要以省份开头
data[data['road'].str.contains('山东')].sample(5)

,city,road,province_short
112037,无锡,香山东路,江苏
2463,北京,房山东大街,北京
87399,上海,中山东二路,上海
68049,宁波,教场山东路,浙江
88613,沈阳,十三纬路山东庙巷,辽宁


In [28]:
data['prov_ext'] = data['road'].str.extract(r"\b({})".format(province_join))[0]

In [29]:
data[data['prov_ext']=='河南'].sample(5)

,city,road,province_short,prov_ext
117808,乌鲁木齐,河南东路南二巷,新疆,河南
29612,台中,河南东五街,台湾,河南
29614,台中,河南路,台湾,河南
20127,苗栗,河南街,台湾,河南
29616,台中,河南路２段,台湾,河南


In [33]:
prov_top10 = data['prov_ext'].value_counts()[:10]
prov_top10

北京    61
上海    49
河北    47
河南    46
重庆    41
天津    39
吉林    32
陕西    29
青海    27
广东    25
Name: prov_ext, dtype: int64

In [34]:
from pyecharts import Bar

bar = Bar("路名中出镜率最高的省级行政单位","单位:次", width = 600,height=500)
bar.add("", prov_top10.index, prov_top10.values, is_stack=True,bar_category_gap='40%', 
       xaxis_label_textsize=20, yaxis_label_textsize=20, is_label_show=True,
       is_convert=True, is_yaxis_inverse=True, label_pos='inside')
bar

In [35]:
def get_unique_city_count(data):
    return len(data.unique())

In [36]:
data.groupby('prov_ext')['city'].apply(get_unique_city_count).nlargest(10)

prov_ext
上海    28
北京    28
天津    24
河南    24
河北    21
吉林    19
山东    18
重庆    18
广东    14
青海    14
Name: city, dtype: int64

##### 3.1.2 出镜最多的城市

In [38]:
city_data2 = pd.read_csv('/home/kesci/input/Country_Road5135/citycode2.csv')
city_data2.sample(5)

,中文名,citycode
371,桃园市,0
131,抚州市,794
333,西宁市,971
367,可克达拉市,1999
190,常德市,736


In [39]:
city_data2['中文名'] = city_data2['中文名'].str.replace('市', '')

In [40]:
city_data2.sample(5)

,中文名,citycode
108,宿州,557
308,阿里,897
143,日照,633
105,黄山,559
276,凉山,834


In [41]:
city = list(city_data2['中文名'].unique())
city_join = '|'.join(city)
data['city_ext'] = data['road'].str.extract(r"\b({})".format(city_join))[0]

In [42]:
city_top12 = data['city_ext'].value_counts()[:12]
city_top12

中山    366
朝阳    167
大同    133
文昌    125
长春    103
东方     89
桃园     63
北京     61
兴安     59
南京     58
南昌     54
上海     49
Name: city_ext, dtype: int64

In [43]:
bar = Bar("路名中出镜率最高的城市","单位:次", width = 600,height=500)
bar.add("", city_top12.index, city_top12.values, is_stack=True,bar_category_gap='40%', 
       xaxis_label_textsize=20, yaxis_label_textsize=20, is_label_show=True,
       is_convert=True, is_yaxis_inverse=True, label_pos='inside')
bar

#### 3.2 哪些城市热衷于使用其它城市名作为道路名

In [44]:
data.sample(5)

,city,road,province_short,prov_ext,city_ext
119871,西安,兴中路,陕西,NaN,NaN
14522,高雄,三和路,台湾,NaN,NaN
22505,台中,太平八街,台湾,NaN,NaN
102977,天津,老左营路,天津,NaN,NaN
119563,西安,尚朴路,陕西,NaN,NaN


In [45]:
city_counts = data.groupby('city')['city_ext'].apply(get_unique_city_count).nlargest(15)
city_counts

city
青岛      131
上海       89
台北       69
天津       64
桃园       57
高雄       56
长春       54
屏东       49
沈阳       43
台东       42
台中       42
苏州       41
乌鲁木齐     39
烟台       39
云林       35
Name: city_ext, dtype: int64

In [54]:
bar = Bar("使用最多城市名作为路名的城市","单位:个", width = 800,height=500)
bar.add("", city_counts.index, city_counts.values, is_stack=True,bar_category_gap='40%', 
       xaxis_label_textsize=20, yaxis_label_textsize=16, is_label_show=True,
       is_convert=True, is_yaxis_inverse=True, label_pos='inside')
bar

In [55]:
# 使用最多省名作为路名的省
data.groupby('province_short')['prov_ext'].apply(get_unique_city_count).nlargest(15).to_csv('prov.csv')
data.groupby('province_short')['prov_ext'].apply(get_unique_city_count).nlargest(15)

province_short
山东     34
台湾     26
湖北     19
上海     16
江苏     16
河北     15
河南     15
四川     13
辽宁     13
黑龙江    12
北京     10
吉林     10
新疆     10
西藏     10
广东      9
Name: prov_ext, dtype: int64

#### 3.3 街、道、路、巷、胡同、里有何不同？

In [56]:
data['road_clean'] = data['road'].str.extract(r"(.*[街|道|路|巷|胡同|里])")[0]

In [57]:
data['endings'] = data['road_clean'].str.extract(r".*(街|道|路|巷|胡同|里)")

In [58]:
data['endings'].value_counts()

路     94788
街     36588
巷     10720
道      7504
胡同     1153
里       421
Name: endings, dtype: int64

In [59]:
import requests


url = 'https://restapi.amap.com/v3/geocode/geo?key=你申请的高德地图API Key&address={}&city={}'


def get_latlon(data):
    res = requests.get(url.format(data[5], data[0]))
    if res.json()['status'] == '1':
        item = res.json().get('geocodes')
        if len(item) > 0:
            return item[0].get('location')

In [60]:
# 胡同/里 都在哪里？
hutong = data[data['endings'] == '胡同']

In [ ]:
hutong['latlon'] = hutong.apply(get_latlon, axis=1)
hutong.to_csv('hutong.csv', index=False)

In [ ]:
li = data[data['endings'] == '里']
li['latlon'] = li.apply(get_latlon, axis=1)
li.to_csv('li.csv', index=False)

#### 3.4 各城市都有“建设路”吗？还有哪些具有时代特色的路名？
工业、人民、光明、建设、胜利、文化、自强、朝阳、复兴、和平

In [61]:
data.loc[data['road'].str.contains('建设'), 'city'].unique()

array(['保定', '成都', '沧州', '长治', '长春', '长沙', '常德', '大连', '德州', '鄂州', '佛山',
       '抚州', '广州', '邯郸', '衡水', '哈尔滨', '呼伦贝尔', '杭州', '黄冈', '衡阳', '怀化',
       '海口', '汉中', '海西', '晋中', '焦作', '锦州', '佳木斯', '济宁', '荆州', '江门', '开封',
       '吕梁', '牡丹江', '马鞍山', '绵阳', '南京', '南阳', '南通', '宁波', '南昌', '南充', '内江',
       '平顶山', '秦皇岛', '齐齐哈尔', '青岛', '上海', '沈阳', '石家庄', '商丘', '韶关', '深圳',
       '石嘴山', '天津', '唐山', '通化', '武汉', '温州', '威海', '西安', '邢台', '新乡', '信阳',
       '徐州', '孝感', '湘西', '西宁', '阳江', '永州', '宜宾', '重庆', '张家口', '淄博', '漳州',
       '枣庄', '株洲', '肇庆', '湛江', '泸州', '屏东', '新北'], dtype=object)

In [62]:
len(data.loc[data['road'].str.contains('建设'), 'city'].unique())

82

In [63]:
data['era'] = data['road'].str.extract(r"(工业|人民|光明|建设|胜利|文化|自强|朝阳|复兴|和平)")[0]

In [64]:
era_top10 = data['era'].value_counts()
era_top10

工业    812
文化    257
人民    225
建设    224
和平    222
光明    217
复兴    191
朝阳    174
胜利    169
自强    135
Name: era, dtype: int64

In [65]:
bar = Bar("带有时代特色的路名","单位: 个", width = 600,height=500)
bar.add("", era_top10.index, era_top10.values, is_stack=True,bar_category_gap='40%', 
       xaxis_label_textsize=20, yaxis_label_textsize=16, is_label_show=True,
       is_convert=True, is_yaxis_inverse=True, label_pos='inside', label_color=['#eb3b5a'])
bar

In [ ]:
gongye = data[data['era'] == '工业']
gongye['latlon'] = gongye.apply(get_latlon, axis=1)
gongye.to_csv('gongye.csv', index=False)

In [ ]:
wenhua = data[data['era'] == '文化']
wenhua['latlon'] = wenhua.apply(get_latlon, axis=1)
wenhua.to_csv('wenhua.csv', index=False)

#### 3.5 体现人们美好愿望的路名？
成功、幸福、太平、永兴、兴业、创业、平安、仁爱、阳光、健康

In [66]:
data['wish'] = data['road'].str.extract(r"(成功|幸福|太平|永兴|兴业|创业|平安|仁爱|阳光|健康)")[0]
wish_top10 = data['wish'].value_counts()
wish_top10

太平    175
幸福    165
成功    164
永兴    124
仁爱    122
兴业     92
平安     90
创业     83
健康     77
阳光     76
Name: wish, dtype: int64

In [67]:
bar = Bar("体现人民美好愿望的路名","单位: 个", width = 600,height=500)
bar.add("", wish_top10.index, wish_top10.values, is_stack=True,bar_category_gap='40%', 
       xaxis_label_textsize=20, yaxis_label_textsize=16, is_label_show=True,
       is_convert=True, is_yaxis_inverse=True, label_pos='inside', label_color=['#eb3b5a'])
bar

In [ ]:
wish = data[data['wish'].isin(['太平', '幸福', '成功', '永兴', '仁爱'])]
wish['latlon'] = wish.apply(get_latlon, axis=1)
wish.to_csv('wish.csv', index=False)

#### 3.6 体现区域地理的路名？
- 海、湖、江、河、涌、滘、潮、湾、港、溪、滨、洲、浦、洋、潭、沟、池
- 岭、峰、山、田、泉、坪、埔、堤、坑、坝、屯

In [69]:
data['water'] = data['road'].str.extract(r"(海|湖|江|河|涌|滘|潮|湾|港|溪|滨|洲|浦|洋|潭|沟|池)")[0]
water_top10 = data['water'].value_counts()[:10]
water_top10

河    2702
江    2594
海    2394
湖    2032
港    1279
溪    1119
滨     744
湾     617
洲     505
浦     499
Name: water, dtype: int64

In [73]:
bar = Bar("跟“水”有关的路名","单位: 个", width = 600,height=500)
bar.add("", water_top10.index, water_top10.values, is_stack=True,bar_category_gap='40%', 
       xaxis_label_textsize=20, yaxis_label_textsize=16, is_label_show=True,
       is_convert=True, is_yaxis_inverse=True, label_pos='inside', label_color=['#eb3b5a'])
bar

In [70]:
water = data[data['water'].isin(['海', '湖', '港', '涌', '滘'])]
water['latlon'] = water.apply(get_latlon, axis=1)
water.to_csv('water.csv', index=False)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [72]:
data['land'] = data['road'].str.extract(r"(岭|峰|山|田|坪|埔|堤|坑|坝|屯)")[0]
land_top10 = data['land'].value_counts()
land_top10

山    5164
田     867
岭     699
峰     672
埔     388
坪     387
堤     289
坑     279
屯     234
坝     225
Name: land, dtype: int64

In [74]:
bar = Bar("跟“山/地/堤”有关的路名","单位: 个", width = 600,height=500)
bar.add("", land_top10.index, land_top10.values, is_stack=True,bar_category_gap='40%', 
       xaxis_label_textsize=20, yaxis_label_textsize=16, is_label_show=True,
       is_convert=True, is_yaxis_inverse=True, label_pos='inside', label_color=['#eb3b5a'])
bar

In [75]:
land = data[data['land'].isin(['田', '岭', '埔', '坑', '屯'])]
land['latlon'] = land.apply(get_latlon, axis=1)
land.to_csv('land.csv', index=False)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


#### 3.7 体现儒释道特色的路名？
塔、庙、寺、仙、台、亭、圣、观、灵、宫

In [76]:
data['religion'] = data['road'].str.extract(r"(塔|庙|寺|仙|台|亭|圣|观|灵|宫)")[0]
religion_top10 = data['religion'].value_counts()
religion_top10

台    572
塔    430
庙    426
仙    411
亭    397
寺    377
圣    372
观    305
灵    295
宫    190
Name: religion, dtype: int64

In [77]:
bar = Bar("受儒释道文化影响的路名","单位: 个", width = 600,height=500)
bar.add("", religion_top10.index, religion_top10.values, is_stack=True,bar_category_gap='40%', 
       xaxis_label_textsize=20, yaxis_label_textsize=16, is_label_show=True,
       is_convert=True, is_yaxis_inverse=True, label_pos='inside', label_color=['#eb3b5a'])
bar

In [82]:
religion = data[data['religion'].isin(['塔', '庙', '仙', '寺', '圣'])]
religion['latlon'] = religion.apply(get_latlon, axis=1)
religion.to_csv('religion.csv', index=False)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


#### 3.8 体现周围景观的路名？
- 林、梅、竹、杨、柳、莲、桃、李、杏、棉

In [83]:
data['plant'] = data['road'].str.extract(r"(林|梅|竹|杨|柳|莲|桃|李|杏|棉)")[0]
plant_top10 = data['plant'].value_counts()
plant_top10

林    1505
竹     632
杨     581
梅     492
柳     423
莲     403
桃     330
李     216
杏     156
棉      65
Name: plant, dtype: int64

In [84]:
bar = Bar("体现周围景观的路名","单位: 个", width = 600,height=500)
bar.add("", plant_top10.index, plant_top10.values, is_stack=True,bar_category_gap='40%', 
       xaxis_label_textsize=20, yaxis_label_textsize=16, is_label_show=True,
       is_convert=True, is_yaxis_inverse=True, label_pos='inside', label_color=['#eb3b5a'])
bar

In [85]:
plant = data[data['plant'].isin(['竹', '杨', '梅', '柳', '莲', '桃', '李', '杏', '棉'])]
plant['latlon'] = plant.apply(get_latlon, axis=1)
plant.to_csv('plant.csv', index=False)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
